In [1]:
import json
import cv2
import os
import shutil

In [2]:
#json_dir = "D:\\Github\\DeepMimic\\Output\\data\\";
json_dir = "/media/yalu/6066C1DD66C1B3D6/Github/DeepMimic/Output/data/"

train_start_id = 2
train_end_id   = 1200
test_start_id  = train_end_id
test_end_id    = 1400

In [3]:
coco_name = ["nose","left_eye","right_eye","left_ear","right_ear",
              "left_shoulder","right_shoulder","left_elbow","right_elbow",
              "left_wrist","right_wrist","left_hip","right_hip",
              "left_knee","right_knee","left_ankle","right_ankle"]
fake_name = ["neck","left_eye","right_eye","left_ear","right_ear",
              "joint_left_shoulder","joint_right_shoulder",
              "joint_left_elbow","joint_right_elbow",
              "left_wrist","right_wrist",
              "joint_left_hip","joint_right_hip",
              "joint_left_knee","joint_right_knee",
              "joint_left_ankle","joint_right_ankle"]
def get_keypoint(data):
    key_point = list()
    for i in range(17):
        if(i > 0 and i < 5):
            key_point.append(0)
            key_point.append(0)
            key_point.append(0)
        else:
            point_deep_name = fake_name[i]
            x = int(data['ScreenPos'][point_deep_name]['x'])          
            y = int(data['ScreenPos'][point_deep_name]['y'])
            if(x > 640 or y > 480):
                view =  0
            else:
                view = data['ScreenPos'][point_deep_name]['view']
            key_point.append(x)
            key_point.append(y)
            key_point.append(view)
    return key_point

In [4]:
#get real tag key point num
def get_keypoint_num(keypoint):
    keypoint_num = 0
    for x in keypoint[2::3]:
        if(x != 0):
            keypoint_num += 1
    return keypoint,keypoint_num

In [5]:
def get_annotation(data,image_id):
    annotation = {}
    bbox = list()
    bbox.append(data['bbox']['x'])
    bbox.append(data['bbox']['y'])
    bbox.append(data['bbox']['width'])
    bbox.append(data['bbox']['height'])
    annotation['area'] = data['bbox']['width'] * data['bbox']['height']
    annotation['id'] = image_id
    annotation['bbox'] = bbox
    annotation['category_id'] = 1
    annotation['image_id'] = image_id
    annotation['iscrowd'] = 0
    annotation['keypoints'] = get_keypoint(data)
    annotation['num_keypoints'] = get_keypoint_num(annotation['keypoints'])
    annotation['segmentation'] = 0
    return annotation

In [6]:
def get_image(data,image_id):
    image = {}
    image['date_captured'] = 0
    image['flickr_url'] = 0
    image['file_name'] = str(image_id) + ".jpg"
    if (image_id < train_end_id):
        image['coco_url'] = "http://images.cocodataset.org/train2017/" + image['file_name']
    elif(image_id < test_end_id):
        image['coco_url'] = "http://images.cocodataset.org/val2017/" + image['file_name']
    else:
        print("Error!")
    image['height'] = 480
    image['width'] = 640
    image['id'] = image_id
    image['license'] = 4
    return image

In [7]:
def parse_deepmimic(image_id):
    json_id = image_id
    json_path = json_dir + str(json_id) + ".json"
    with open(json_path) as json_file:  
        data = json.load(json_file)
    annotation = get_annotation(data,image_id)
    image = get_image(data,image_id)
    return annotation,image

In [8]:
categorie = {
    "supercategory": "person",
    "id": 1,
    "name": "person",
    "keypoints": ["nose","left_eye","right_eye","left_ear","right_ear","left_shoulder","right_shoulder","left_elbow","right_elbow","left_wrist","right_wrist","left_hip","right_hip","left_knee","right_knee","left_ankle","right_ankle"],
    "skeleton": [[16,14],[14,12],[17,15],[15,13],[12,13],[6,12],[7,13],[6,7],[6,8],[7,9],[8,10],[9,11],[2,3],[1,2],[1,3],[2,4],[3,5],[4,6],[5,7]]
}

In [9]:
def get_json_data(start,end):
    annotations = list()
    images = list()
    categories = list() 
    for image_id in range(start,end):
        annotation,image = parse_deepmimic(image_id)
        annotations.append(annotation)
        images.append(image)
    categories.append(categorie)
    coco_file = {}
    coco_file["annotations"] = annotations
    coco_file["images"] = images
    coco_file["categories"] = categories
    return coco_file

In [10]:
coco_write_dir = "/home/yalu/data/deepmimic"

In [11]:
json_write_path= coco_write_dir  + "/annotations/person_keypoints_train2017.json"
json_data = get_json_data(train_start_id,train_end_id)
with open(json_write_path, "w") as json_file:  
    json.dump(json_data,json_file)

In [12]:
json_write_path = coco_write_dir  + "/annotations/person_keypoints_val2017.json"
json_data = get_json_data(test_start_id,test_end_id)
with open(json_write_path, "w") as json_file:  
    json.dump(json_data,json_file)

In [13]:
coco_data_dir = coco_write_dir + "/train2017"
if not os.path.exists(coco_data_dir):
    os.makedirs(coco_data_dir)

In [14]:
coco_data_dir = coco_write_dir + "/val2017"
if not os.path.exists(coco_data_dir):
    os.makedirs(coco_data_dir)

In [15]:
image_src_dir = "/media/yalu/6066C1DD66C1B3D6/Github/DeepMimic/Output/image_without_point/"

In [16]:
for image_id in range(train_start_id,train_end_id):
    image_src = image_src_dir + str(image_id) + ".jpg"
    image_dst = coco_write_dir + "/train2017/" + str(image_id) + ".jpg"
    shutil.copy(image_src,image_dst)

In [17]:
for image_id in range(test_start_id,test_end_id):
    image_src = image_src_dir + str(image_id) + ".jpg"
    image_dst = coco_write_dir + "/val2017/" + str(image_id) + ".jpg"
    shutil.copy(image_src,image_dst)